<a href="https://colab.research.google.com/github/Deepika1454/Deepikaravi/blob/main/twitter_sentimental_analysis_by_Deepika_R_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing kaggle library
! pip install kaggle

In [ ]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('the dataset is extracted')


the dataset is extracted


importing the dependencies

In [ ]:
  import numpy as pd
  import pandas as pd
  import re
  from nltk.corpus import stopwords
  from nltk.stem.porter import PorterStemmer
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.model_selection import train_test_split
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data processing

In [ ]:
#loading the data from csv file to pandas dataFrame
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [ ]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#printing the first 5 rows of the dataFrame
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the column and reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'tweet']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1', names = column_names)

In [ ]:
# checking the number of rows and columns
twitter_data.shape


(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting the number of misssing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
tweet     0
dtype: int64

all values are present

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert the target "4" to "1"

In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Negative Tweet

**Stemming**
          stemming is the process of reudcing a word to its root word and only words
        not symbols

ex: root words like actor, actress, acting = act

In [ ]:
port_steam = PorterStemmer()

In [ ]:
   def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_steam.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data['tweet'] = twitter_data['tweet'].apply(stemming) # wait for 50 min

In [ ]:
twitter_data.head()

,target,id,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see


In [ ]:
print(twitter_data['tweet'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: tweet, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# seperating the data and label
X = twitter_data['tweet'].values
Y = twitter_data['target'].values

In [ ]:
print(X) # first 2 and last 2 tweet

In [ ]:
print(Y) # first 2 and last 2 target

splitting the data to training data and testing data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=y, random_state = 2) # stratify is equal propulsion for both test and train

In [ ]:
print(X.shape, x_train.shape, x_test.shape)

In [ ]:
print(x_train)

In [ ]:
print(x_test)

In [ ]:
# converting textual data to numerical data

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

In [ ]:
print(x_test)

training the machine learning model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000) #Maximum iteration is max number of times the model can go through the data

In [ ]:
model.fit(x_train, y_train)

Model Evaluation

accuracy **score**

In [ ]:
# accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy) # so the model is 81 percent accurate

here in accuracy score our model can predicted 81% **correctly** remaininng 19% it predicted **wrongly** . this is what accuracy score

have to see the test data

In [ ]:
# accuracy score on the test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy) # so the model is 77 percent accurate

MODEL ACCURACY**77% correctly in test and in training 81% correctly predicted , it is very close to each other so we can say the model has performed well**

*and it is not overfitted which means not having much difference in training and testing*

**SAVING THE TRAINED** **MODEL**

In [ ]:
import pickle


In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) #wb is writing a file in binary format

using the saved modelfor future predictions

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
x_new = x_test[200]
print(y_test[200])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('the tweet is negative')
else:
  print('the tweet is positive')

In [ ]:
x_new = x_test[3]
print(y_test[3])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('the tweet is negative')
else:
  print('the tweet is positive')

In [ ]:
x_new = x_test[322]
print(y_test[322])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('the tweet is negative')
else:
  print('the tweet is positive')